In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shapely.geometry import Polygon, Point, LinearRing
import seaborn as sns; sns.set()
import math
%matplotlib notebook

In [2]:
poly = Polygon([(3750901.5068, -19268905.6133), (3750901.5068,-19208905.6133),
                (3770901.5068, -19268905.6133), (3770901.5068,-19208905.6133)])

def city_distance(x, y):
    point = Point(x, y)
    pol_ext = LinearRing(poly.exterior.coords)
    d = pol_ext.project(point)
    p = pol_ext.interpolate(d)
    closest_point_coords = list(p.coords)[0]
    return point.distance(Point(closest_point_coords))

dd_1 = dd[(dd['duration'] != 0.0) & (dd['x_exit'] >= 3750901.5068) & 
         (dd['x_exit'] <= 3770901.5068) & (dd['y_exit'] >= -19268905.6133) & 
         (dd['y_exit'] <= -19208905.6133)]

In [ ]:
df_train = pd.read_csv("../../data/data_train.csv")
df_test = pd.read_csv("../../data/data_test.csv")
df_train['time_entry'] = pd.to_timedelta(df_train['time_entry'])
df_train['time_exit'] = pd.to_timedelta(df_train['time_exit'])
df_test['time_entry'] = pd.to_timedelta(df_test['time_entry'])
df_test['time_exit'] = pd.to_timedelta(df_test['time_exit'])
df_train['duration'] = df_train['time_exit'] - df_train['time_entry']
df_train['duration'] = df_train['duration'].dt.total_seconds()
df_test['duration'] = df_test['time_exit'] - df_test['time_entry']
df_test['duration'] = df_test['duration'].dt.total_seconds()
df_train['distance'] = df_train.apply(lambda row: Point(row['x_entry'],row['y_entry']).
                                      distance(Point(row['x_exit'],row['y_exit'])), axis=1)
df_train['velocity'] = df_train.apply(lambda row: (row['distance']/row['duration'])
                                      if row['duration']!= 0.0 else 0.0, axis=1)
df_test['city_distance'] = df_test.apply(lambda row: 0 if ((row['x_entry'] >= 3750901.5068) &
                                                    (row['x_entry'] <= 3770901.5068) & 
                                                    (row['y_entry'] >= -19268905.6133) & 
                                                    (row['y_entry'] <= -19208905.6133)) 
                                  else city_distance(row['x_entry'], row['y_entry']), axis = 1)
df_train['city_distance'] = df_train.apply(lambda row: 0 if ((row['x_entry'] >= 3750901.5068) &
                                                    (row['x_entry'] <= 3770901.5068) & 
                                                    (row['y_entry'] >= -19268905.6133) & 
                                                    (row['y_entry'] <= -19208905.6133)) 
                                  else city_distance(row['x_entry'], row['y_entry']), axis = 1)
df_train['ratio'] = df_train.apply(lambda row: (row['city_distance']/row['duration'])
                                      if row['duration']!= 0.0 else 0.0, axis=1)
df_test['ratio'] = df_test.apply(lambda row: (row['city_distance']/row['duration'])
                                      if row['duration']!= 0.0 else 0.0, axis=1)
df_train['label']= -1
df_train['label'] = df_train.apply(lambda row: 1 
                                 if ((row['x_exit'] >= 3750901.5068) & (row['x_exit'] <= 3770901.5068) & 
                                     (row['y_exit'] >= -19268905.6133) & (row['y_exit'] <= -19208905.6133)) 
                                 else 0, axis=1)
df_test['label']= -1
df_test['label'] = df_test.apply(lambda row: 1 
                                 if ((row['duration'] == 0.0)&(math.isnan(row['x_exit'])) & (math.isnan(row['y_exit'])) &
                                      (row['x_entry'] >= 3750901.5068) & (row['x_entry'] <= 3770901.5068) & 
                                      (row['y_entry'] >= -19268905.6133) & (row['y_entry'] <= -19208905.6133)) 
                                 else row['label'], axis=1)
df_test['label'] = df_test.apply(lambda row: 0 
                                 if ((row['duration'] == 0.0)&(math.isnan(row['x_exit'])) & (math.isnan(row['y_exit'])) &
                                     (row['label'] == -1))
                                 else row['label'], axis=1)

In [ ]:
test = df_test[(df_test['duration'] != 0.0) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())]
start_time = '0 days 15:00:00'
end_time = '0 days 16:00:00'
dd = df_train[(df_train['time_exit'] >= start_time) & (df_train['time_exit'] <= end_time)]
dd[(dd['duration'] == 0.0) & (dd['x_exit'] >= 3750901.5068) & 
         (dd['x_exit'] <= 3770901.5068) & (dd['y_exit'] >= -19268905.6133) & 
         (dd['y_exit'] <= -19208905.6133)].label.value_counts()
dd[(dd['duration'] == 0.0) & ((dd['x_exit'] < 3750901.5068) | 
         (dd['x_exit'] > 3770901.5068)) & ((dd['y_exit'] < -19268905.6133) | 
         (dd['y_exit'] > -19208905.6133))].label.value_counts()
dd_1 = dd[(dd['duration'] != 0.0) & (dd['x_exit'] >= 3750901.5068) & 
         (dd['x_exit'] <= 3770901.5068) & (dd['y_exit'] >= -19268905.6133) & 
         (dd['y_exit'] <= -19208905.6133)]
train = dd[dd['duration'] != 0.0]

In [ ]:
df = df_train.groupby('hash')
df_list = list(df)
df_2 = df_test.groupby('hash')
df_2_list = list(df_2)

In [ ]:
tt = df_train[(df_train['x_exit'] >= 3750901.5068) & (df_train['x_exit'] <= 3770901.5068) & 
              (df_train['y_exit'] >= -19268905.6133) & (df_train['y_exit'] <= -19208905.6133)]
tt2 = df_test[(df_test['x_exit'] >= 3750901.5068) & (df_test['x_exit'] <= 3770901.5068) & 
              (df_test['y_exit'] >= -19268905.6133) & (df_test['y_exit'] <= -19208905.6133)]

# Plottings

In [ ]:
def plot_trajectory(df_sample):
    rect = patches.Rectangle((3750901.5068,-19208905.6133), (3770901.5068 - 3750901.5068),
                             (- 19268905.6133 + 19208905.6133),
                             linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city center')
    city = plt.gca().add_patch(rect)
    for index, row in df_sample.iterrows():
#         lines = plt.plot(df_sample.x_exit,df_sample.y_exit, label='Lines', color = 'g')
        arrows = plt.arrow(row['x_entry'], row['y_entry'], row['x_exit'] - row['x_entry'], row['y_exit'] - row['y_entry'],
                  label ='Trajectory', color = 'b')
    entries = plt.scatter(df_sample.x_entry,df_sample.y_entry, label='Entry point', color = 'b')
    exits = plt.scatter(df_sample.x_exit,df_sample.y_exit, label='Exit point', color = 'r')
#     lines = plt.plot(df_sample.x_exit,df_sample.y_exit, label='Lines', color = 'g')
    plt.legend(handles=[city, arrows])
    plt.grid(True)
    plt.show()

In [ ]:
def plot_fig(train):
    rect = patches.Rectangle((3750901.5068,-19208905.6133), (3770901.5068 - 3750901.5068),
                                 (- 19268905.6133 + 19208905.6133),
                                 linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city center')
    entries = plt.scatter(train.x_entry,train.y_entry, label='Entry point', color = 'b', s = 0.0002)
#     exits = plt.scatter(train.x_exit,train.y_exit, label='Exit point', color = 'r', s = 0.00002)
    for index, row in train.iterrows():
        if(row['label'] == 0):
            arrows1 = plt.arrow(row['x_entry'], row['y_entry'], row['x_exit'] - row['x_entry'], row['y_exit'] - row['y_entry'],
                      label ='Not in City', color = 'b', linewidth = 0.001)
#         else:
#             arrows2 = plt.arrow(row['x_entry'], row['y_entry'], row['x_exit'] - row['x_entry'], row['y_exit'] - row['y_entry'],
#                       label ='In City', color = 'r', linewidth = 0.00005)
    city = plt.gca().add_patch(rect)
    plt.legend(handles = [arrows1, city, entries])
    plt.title("Not in city journeys")
    plt.grid(True)
    plt.savefig('foo.png', dpi=1200)
    plt.show()

In [ ]:
def plot_test_fig(sample):
    rect = patches.Rectangle((3750901.5068,-19208905.6133), (3770901.5068 - 3750901.5068),
                                 (- 19268905.6133 + 19208905.6133),
                                 linewidth=1,edgecolor='g',fill = False,hatch = '\\\\\\', label = 'city center')
    entries = plt.scatter(sample.x_entry,sample.y_entry, label='Entry point', color = 'b', s = 0.002)
    city = plt.gca().add_patch(rect)
    plt.legend(handles = [city,entries])
    plt.title("All test starting point in map")
    plt.grid(True)
    plt.savefig('foo1.png', dpi=1200)
    plt.show()

In [ ]:
plot_trajectory(df_list[100][1])

# Test Explore

In [ ]:
df_test[(df_test.x_exit.isnull()) & (df_test.y_exit.isnull())].duration.describe()

In [ ]:
df_test[(df_test['duration'] > 0) & (df_test['duration'] < 5000) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())].duration.describe()

In [ ]:
plt.scatter(df_train[df_train['velocity'] != 0.0].distance, df_train[df_train['velocity'] != 0.0].duration)

In [ ]:
df_test[df_test.x_exit.isnull()]['label'].value_counts()

In [ ]:
df_test['label'] = df_test['label'].replace(-1,0)

In [ ]:
df_test[df_test.x_exit.isnull()][['trajectory_id', 'label']].to_csv('output.csv', index=False)

# Train explore

In [ ]:
df_train[(df_train['duration'] == 0.0) & ((df_train['x_entry'] != df_train['x_exit']) |
         (df_train['y_entry'] != df_train['y_exit']))]

In [ ]:
df_train[(df_train['duration'] == 0.0) & (df_train['x_entry'] == df_train['x_exit']) &
         (df_train['y_entry'] == df_train['y_exit'])].shape[0]

In [ ]:
df_train.label.value_counts()

In [ ]:
df_train[(df_train['duration'] == 0.0) & (df_train['x_exit'] >= 3750901.5068) & 
         (df_train['x_exit'] <= 3770901.5068) & (df_train['y_exit'] >= -19268905.6133) & 
         (df_train['y_exit'] <= -19208905.6133)]

# Explorations

In [ ]:
ax = sns.scatterplot(x="duration", y="city_distance", hue = 'label', data=dd[(dd['duration']!=0)&(dd['label'] == 1)])

In [ ]:
plt.clf()
plot_trajectory(dd_1)

In [ ]:
plt.clf()
plot_fig(train[train['label'] == 0])

In [ ]:
plt.clf()
plot_test_fig(df_test[(df_test['duration'] > 0) & (df_test.x_exit.isnull()) & (df_test.y_exit.isnull())])

In [ ]:
plt.scatter(df_train[df_train['duration']!= 0.0].duration, df_train[df_train['duration']!= 0.0].distance,
            label='Entry point', color = 'b', s= 0.01)

In [ ]:
plt.clf()
df_train[(df_train['velocity'] != 0.0)&(df_train['velocity'] < 100.0)].velocity.hist()

In [ ]:
plt.scatter(test.city_distance, test.duration, color = 'b', s = 0.01)

In [ ]:
plt.clf()
plt.scatter(train[train['label'] == 0].city_distance, train[train['label'] == 0].duration, color = 'r', s = 0.0001)
plt.scatter(train[train['label'] == 1].city_distance, train[train['label'] == 1].duration, color = 'b', s = 0.0001)

In [ ]:
ax = sns.scatterplot(x="city_distance", y="duration", hue = 'label', data=train, s=5)